<a href="https://colab.research.google.com/github/nehabindle/NehaBindle/blob/master/YelpNeha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s gdrive/'Team Drives'/'Data Mining Team'/ gdata

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls gdata/

'Preprocessing link.gdoc'   yelpDataset.ipynb	      yelpHotelData.db
'Project proposal.gdoc'     yelpDataset_kunj.ipynb    YelpNeha.ipynb
'Untitled document.gdoc'    yelpDataset_shrey.ipynb


In [4]:
!pip install contractions

In [0]:
import numpy as np
import pandas as pd
import nltk
import spacy
from tqdm import tqdm
import os
import seaborn as sns
import matplotlib.pyplot as plt
import re, string, unicodedata
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [0]:
import sqlite3
import pandas as pd
connection = sqlite3.connect('gdata/yelpHotelData.db')
x1 = connection.execute("select * FROM review")

In [0]:
description = []
Class = []

data = x1.fetchmany(10000)

for x in data:
  description.append(x[3])
  Class.append(x[8])
trainData = {'Class' : Class, 'Description' : description}
df_X = pd.DataFrame(trainData)

In [8]:
df_X['Class'].value_counts()
#df_X

N     5078
NR    2461
YR    1681
Y      780
Name: Class, dtype: int64

In [0]:
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [10]:
df_X['Class'].value_counts(normalize = True)

N     0.5078
NR    0.2461
YR    0.1681
Y     0.0780
Name: Class, dtype: float64

In [0]:
# remove URL's from train and test
 df_X['clean_desc']= df_X['Description'].apply(lambda x: re.sub(r'http\S+', '', x))


In [0]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~.,'

df_X['clean_desc'] = df_X['clean_desc'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

In [0]:
# convert text to lowercase
df_X['clean_desc'] = df_X['clean_desc'].str.lower()

In [0]:
# remove whitespaces
df_X['clean_desc'] = df_X['clean_desc'].apply(lambda x:' '.join(x.split()))

In [0]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])


In [0]:
# tokenization using spaCy
def tokenization(x):
    desc_tokens = []
    for i in tqdm(x):
        i = nlp(i)
        temp = []
        for j in i:
            temp.append(j.text)
        desc_tokens.append(temp)
    
    return desc_tokens

In [18]:
#Tokenization for Train data
df_X['clean_desc'] = tokenization(df_X['clean_desc'])

100%|██████████| 10000/10000 [01:59<00:00, 83.45it/s]


In [0]:
#function to remove stopwords
def remove_stopwords(desc):
    s = []
    for r in tqdm(desc):
        s_2 = []
        for token in r:
            if nlp.vocab[token].is_stop == True:
                continue
            else:
                s_2.append(token)
        s.append(" ".join(s_2))    
        
    return s

In [20]:
df_X['clean_desc'] = remove_stopwords(df_X['clean_desc'])

100%|██████████| 10000/10000 [00:02<00:00, 4954.73it/s]


In [0]:
# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
             s = [token.lemma_ for token in nlp(i)]
             output.append(' '.join(s))
    return output

In [0]:
df_X['clean_desc'] = lemmatization(df_X['clean_desc'])

In [23]:
df_X.sample(10)

,Class,Description,clean_desc
639,N,I enjoyed my 3 night stay here. The only thing I didn't like about my room was that it didn't have a microwave or a fridge. I mean they provided us with chilled alcohol and water in this mini co...,enjoy 3 night stay thing n't like room n't microwave fridge mean provide chill alcohol water mini cooler look thing candy snack touch lol odd reasoni noticed hilton hotel fan restroom lol love bed...
4981,Y,"Just completed a big family reunion at this property. We had a luncheon buffet and a dinner buffet, and all of the food was excellent. The pastries are to die for. The new restaurant, which opened...",complete big family reunion property luncheon buffet dinner buffet food excellent pastry die new restaurant open month ago extensive menu prior hotel staff attentive need room small bathroom small...
9758,NR,I really like this place. I met my girlfriend and a friend of hers a couple of weeks ago for a drink and quick bite. The location is great as well as they are placed on the corner in front of the...,like place meet girlfriend friend couple week ago drink quick bite location great place corner movie theater garage bay door open large airy spacious price high right line area dollar compare esta...
2449,N,"The Hilton is one of those Grand Old Ladies that has been preserved with love and attention. I booked with Priceline for $79 (Name Your Own Price) as opposed to paying $225 per night. At $225, I'd...",hilton grand old lady preserve love attention book priceline 79 price oppose pay 225 night 225 'd annoy 79 ' m delighted staff marginal desk clerk mildly snotty concierge n't answer call helpful p...
9795,NR,"Last night after some dinner, the kids wanted to stop at BlissKiwi (frozen yogurt). I had never heard of the place, but yet it was so close to home. Apparently they have only been open for a shor...",night dinner kid want stop blisskiwi frozen yogurt hear place close home apparently open short pull 745 place pack know attendance place instant success walk bright color trendy decor express fun ...
5265,Y,"This hotel was exactly like you would think, the regular accommodations but i wish it were a bit closer to good food and bars. The service was great, and Spa was very clean.",hotel exactly like think regular accommodation wish bit close good food bar service great spa clean
5119,Y,Definitely NOT worth paying full price; I Hotwired this hotel for $175/night (about half off) and it wasn't even worth that. Hotwire had it listed as 4-4.5 stars and while it looks fancy in the lo...,definitely worth pay price hotwir hotel 175night half n't worth hotwire list 445 star look fancy lobby outside room bed fullsize good light terrible fridgemini bar situation nightmare afraid touch...
1570,N,"Hands down one of the nicest hotels in Chicago I have ever stayed at. Service was wonderful, location is great! Room was so comfortable, and had a wonderful view of the Chicago River, workout ar...",hand nice hotel chicago stay service wonderful location great room comfortable wonderful view chicago river workout area modern great view stay reason come price bad chicago book hotwire
6331,NR,"Love the Santa Ana Drag Strip margaritas, Ramses make the best. I wish it was not so close to my home, I seem to gravitate towards it between 5-7 everyday, great happy hour. The Bar B Q there is g...",love santa ana drag strip margaritas rams good wish close home gravitate 57 everyday great happy hour bar b q great serve nice waitress christina love place
9062,NR,Dead on cajun perfection! Great food & perfect service! An absolute must experience!,dead cajun perfection great food perfect service absolute experience


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [0]:
# build TF-IDF features for train data
cv = TfidfVectorizer(use_idf=True, min_df=5, max_df=0.4, ngram_range=(1,2),
                        sublinear_tf=True,)
cv_train = cv.fit_transform(df_X['clean_desc'])

In [30]:
cv_train

<10000x22811 sparse matrix of type '<class 'numpy.float64'>'
	with 695681 stored elements in Compressed Sparse Row format>

Training the model using SVM

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(cv_train,df_X['Class'], test_size=0.3, random_state=42)

In [33]:
X_train.shape

(7000, 22811)

In [34]:
X_test.shape

(3000, 22811)

In [35]:
y_train.shape

(7000,)

In [36]:
y_test.shape

(3000,)

In [0]:
from sklearn import svm

In [38]:
clf = svm.SVC(gamma='scale', decision_function_shape='ovo')

clf.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [39]:
from sklearn import metrics
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           N       0.82      0.98      0.89      1551
          NR       0.62      0.84      0.72       746
           Y       0.00      0.00      0.00       240
          YR       0.63      0.19      0.29       463

   micro avg       0.74      0.74      0.74      3000
   macro avg       0.52      0.50      0.48      3000
weighted avg       0.68      0.74      0.68      3000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
